In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']
openai.api_type = os.environ['OPENAI_API_TYPE']
openai.api_version = os.environ['OPENAI_API_VERSION']

os.environ['LANGCHAIN_PROJECT'] = 'customizing_run_names'

In [2]:
from langsmith import Client
client = Client()

In [3]:
from langchain import prompts, callbacks, schema
from langchain.chat_models import AzureChatOpenAI

model = AzureChatOpenAI(temperature=0.0,
    openai_api_base=openai.api_base,
    openai_api_version=openai.api_version,
    deployment_name="gpt-35-turbo-16k",
    openai_api_key=openai.api_key,
    openai_api_type = openai.api_type,
)

chain = (
    prompts.ChatPromptTemplate.from_template("Reverse the following string: {text}")
    | model
).with_config({"run_name": "StringReverse"})

with callbacks.collect_runs() as cb:
    for chunk in chain.stream({"text": "🌑🌒🌓🌔🌕"}):
        print(chunk.content, flush=True, end="")
    run = cb.traced_runs[0]

🌕🌔🌓🌒🌑

In [6]:
run.name

'StringReverse'

In [7]:
run.outputs

{'output': AIMessageChunk(content='🌕🌔🌓🌒🌑')}

In [8]:
# List with the name filter to get runs with the assigned name
next(client.list_runs(project_name=os.environ['LANGCHAIN_PROJECT'], filter='eq(name, "StringReverse")'))

Run(id=UUID('6a802275-a598-461f-98c1-966bf902c25c'), name='StringReverse', start_time=datetime.datetime(2023, 12, 4, 3, 25, 26, 160888), run_type='chain', end_time=datetime.datetime(2023, 12, 4, 3, 25, 28, 311379), extra={'runtime': {'cpu': {'time': {'sys': 0.546875, 'user': 1.40625}, 'percent': 0.0, 'ctx_switches': {'voluntary': 11191.0, 'involuntary': 0.0}}, 'mem': {'rss': 170450944.0}, 'library': 'langchain', 'runtime': 'python', 'platform': 'Windows-10-10.0.22621-SP0', 'sdk_version': '0.0.54', 'thread_count': 27.0, 'library_version': '0.0.327', 'runtime_version': '3.10.12', 'langchain_version': '0.0.327', 'py_implementation': 'CPython'}}, error=None, serialized=None, events=[{'name': 'start', 'time': '2023-12-04T03:25:26.160888'}, {'name': 'end', 'time': '2023-12-04T03:25:28.311379'}], inputs={'text': '🌑🌒🌓🌔🌕'}, outputs={'output': {'type': 'AIMessageChunk', 'content': '🌕🌔🌓🌒🌑', 'example': False, 'additional_kwargs': {}}}, reference_example_id=None, parent_run_id=None, tags=[], execut

## With Agents

In [9]:
from langchain import agents, tools


agent_executor = agents.initialize_agent(
    llm=model,
    tools=[tools.ReadFileTool(), tools.WriteFileTool(), tools.ListDirectoryTool()],
    agent=agents.AgentType.OPENAI_FUNCTIONS,
)

In [10]:
with callbacks.collect_runs() as cb:
    result = agent_executor.with_config({"run_name": "File Agent"}).invoke("What files are in the current directory?")
    run = cb.traced_runs[0]
    print(result['output'])

The files in the current directory are:
1. Customizing_Run_Names.ipynb
2. Tracing_Nested_Calls_within_Tools.ipynb


In [11]:
callbacks.tracers.langchain.wait_for_all_tracers()
print(f"Saved name: {run.name}")

Saved name: File Agent


### An easy way to customize run names is to use the with_config syntax on your LangChain chain or runnable lambda. This makes it easier to understand a trace at a glance.

In [12]:
# Get trace link
from langchain.callbacks import tracing_v2_enabled

with tracing_v2_enabled() as cb:
    agent_executor.invoke({"What files are in the current directory?"})
    url = cb.get_run_url()

print(f"Trace link: {url}")

Trace link: https://smith.langchain.com/o/9da5a34e-f5e5-5974-bc8b-2c86a11d43cf/projects/p/54ea1e16-85f8-4cd9-a2b7-4085a210240f/r/9531b1f4-a03d-42e9-985b-3b48dac6eb18?poll=true
